In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from keras.models import load_model

In [3]:
#import CSV to be read as a pandas dataframe
labels = pd.read_csv('/content/drive/MyDrive/Preprocessed_BRCA_RNA_data_normalized.csv', header = 0)
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Encoded_SDAEdata.csv', header = None)


#encode categorical variables to numerical
labelenc = LabelEncoder()
labels['Class'] = labelenc.fit_transform(labels['Class'])

print(data)

#get labels from original data and use encoded data as x
x = data
y = labels['Class']
print(y)

print(x.shape)
print(y.shape)

      0           1     2     3     4     ...  7995  7996  7997  7998        7999
0      0.0  35495720.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  21416834.0
1      0.0  36778148.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  22191268.0
2      0.0  36375916.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  21948170.0
3      0.0  40094680.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  24193660.0
4      0.0  38642548.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  23317530.0
...    ...         ...   ...   ...   ...  ...   ...   ...   ...   ...         ...
1090   0.0  38656424.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  23324048.0
1091   0.0  36256928.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  21876094.0
1092   0.0  39195424.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  23651812.0
1093   0.0  39822172.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  24026824.0
1094   0.0  36898480.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0  22263468.0

[1095 rows x 80

In [4]:
#divide into train and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(876, 8000)
(219, 8000)
(876,)
(219,)


In [5]:
#scale data & normalize
t = MinMaxScaler()
t.fit(x_train)
t.fit(x_test)

x_train = t.transform(x_train)
x_test = t.transform(x_test)

In [6]:
#set classifier to vote of 3 neighbors, weighted by distance from k
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')


In [7]:
#Gets validations stats iterated from every fold 
pred_dict = []
def cscorer(clf, x, y):
  y_pred = clf.predict(x)
  metric = metrics.classification_report(y, y_pred)
  pred_dict.append(metric)
  return 1

all_pred = cross_val_score(knn, x, y, cv=5, scoring= cscorer)


for i in pred_dict:
  print('5-fold cross validation accuracies are: ')
  print(i)

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.00      0.00      0.00        23
           2       0.80      0.94      0.86       174

    accuracy                           0.74       219
   macro avg       0.27      0.31      0.29       219
weighted avg       0.63      0.74      0.68       219

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.11      0.05      0.06        22
           1       0.33      0.13      0.19        23
           2       0.80      0.93      0.86       174

    accuracy                           0.75       219
   macro avg       0.42      0.37      0.37       219
weighted avg       0.68      0.75      0.71       219

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
      